In [105]:
import numpy as np
import pandas as pd
from pandas import json_normalize

from bs4 import BeautifulSoup
import requests 
import time

from selenium import webdriver 
from selenium.webdriver.common.keys import Keys 
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from geopy.geocoders import Nominatim

import warnings
warnings.filterwarnings("ignore")

In [106]:
# pip install geopy

# **Locations Wikipedia scraping**

## Testing

In [141]:
df = pd.read_csv('Datasets/df_ratings.csv')
df.shape

(76742, 5)

In [108]:
df.sample(5)

,album_id,artist,album,rating,votes
41115,173261,Oh The Places You'll Go,Pleasure Is A Burden,1.00,1
75604,223276,Demi Semi Quaver,II,4.00,1
64756,207609,prof.logik,Dirt Vol. 3,2.00,2
724,971,Chevelle,This Type of Thinking (Could Do Us In),3.79,1576
59538,200115,Sin Orden,Ha Llegado El Momento,3.50,1


In [125]:
artists = df['artist'].unique()
len(artists)

9616

In [111]:
artists[8]

'Rooney'

In [115]:
artist = "Rooney"
name_changed = artist.replace(' ', '_')
name_changed_band = artist.replace(' ', '_') + ('_(band)')
url = f"https://en.wikipedia.org/wiki/{name_changed_band}"

response = requests.get(url).content
soup = BeautifulSoup(response, "html.parser")
origin = soup.select('table tr th', class_='infobox-label')[2].text
location = soup.select('table tr td', class_='infobox-data')[1].text
location


'This article needs additional citations for verification. Please help improve this article by adding citations to reliable sources. Unsourced material may be challenged and removed.Find sources:\xa0"Rooney"\xa0band\xa0–\xa0news\xa0· newspapers\xa0· books\xa0· scholar\xa0· JSTOR (March 2011) (Learn how and when to remove this message)'

In [116]:
# 0 - Less_Than_Jake_(band): Gainesville, Florida, U.S.
# 1 - Sparta_(band): El Paso, Texas
# 2 - Muse_(band): Teignmouth, Devon, England
# 3 - Finch: no location found
# 4 - Queens_of_the_Stone_Age: Seattle, Washington, U.S.
# 5 - Transplants_(band): additional citations - Los Angeles, California, United States[1]
# 6 - Dream_Theater_(band): Boston, Massachusetts, United States
# 7 - A_Perfect_Circle: Los Angeles, California, U.S.
# 8 - Rooney_(band): additional citations - Los Angeles, California, U.S.
# 9 - AFI_(band): Ukiah, California, U.S.
# 10 - Sublime_(band): Long Beach, California, U.S.
# 11 - The_Cure_(band): Crawley, England
# 12 - blink-182: Poway, California, U.S.
# 13 - Weezer_(band): Los Angeles, California, U.S.
# 14 - Deftones_(band): Sacramento, California, U.S.
# 15 - Jimmy_Eat_World: Mesa, Arizona, U.S.
# 16 - NOFX: Los Angeles, California, U.S.
# 17 - The_Mars_Volta: El Paso, Texas, U.S.
# 18 - Black_Sabbath_(band): Birmingham, England
# 19 - Raised_Fist: Luleå, Norrbotten, Sweden

In [143]:
artist = "Sugarcult"
name_changed = artist.replace(' ', '_')
name_changed_band = artist.replace(' ', '_') + ('_(band)')
url = f"https://en.wikipedia.org/wiki/{name_changed}"

response = requests.get(url).content
soup = BeautifulSoup(response, "html.parser")

# origin = soup.select('table tr th', class_='infobox-label')[2].text
# # location = soup.select('table tr td', class_='infobox-data')[7].text
# location = soup.select('table tr td', class_='infobox-data')[1].text
# location

table = soup.select('#mw-content-text > div.mw-content-ltr.mw-parser-output > table.infobox')

table[0].text.split('Origin')[1].split('Genres')[0]
# table[0].text.split(')')[2].split('Genres')[0]
# table[0].text

'Santa Barbara, California, United States'

In [118]:
df.head()

,album_id,artist,album,rating,votes
0,37,Less Than Jake,Losing Streak,3.90,414
1,40,Sparta,Wiretap Scars,3.79,431
2,41,Muse,Absolution,3.99,4411
3,42,Muse,Showbiz,3.50,2181
4,45,Finch,What It Is to Burn,3.69,864


## Changing values

In [127]:
df[df['artist']=='pg.99 / Majority Rule']

,album_id,artist,album,rating,votes


In [122]:
df['artist'] = np.where(df['artist']=='pg.99 / Majority Rule', 'Majority Rule', df['artist'])
df[df['artist']=='pg.99 / Majority Rule']

,album_id,artist,album,rating,votes


In [ ]:
df.to_csv('Datasets/df_ratings.csv', index=False)

## Other codes

### **CODE**

In [ ]:
df = pd.read_csv('Datasets/df_ratings.csv')
artists = df['artist'].unique()

artists_list = []
origin_list = []
count=0

for index in artists[0:10]:

    artists_list.append(index)
    name_changed = index.replace(' ', '_')
    name_changed_band = name_changed + ('_(band)')

    try:
        url = f"https://en.wikipedia.org/wiki/{name_changed_band}"
        response = requests.get(url).content
        soup = BeautifulSoup(response, "html.parser")

        origin = soup.select('table tr th', class_='infobox-label')

        if len(origin) > 0:
            try:
                if origin[2].text == 'Origin':
                    location = soup.select('table tr td', class_='infobox-data')[1].text
                elif origin[3].text == 'Origin':
                    location = soup.select('table tr td', class_='infobox-data')[2].text
                else:
                    

                if 'This article' in location:
                    location = soup.select('table tr td', class_='infobox-data')[3].text        
                    print(f'{count} - {name_changed_band}: {location}')
                    origin_list.append(location)
                else:
                    print(f'{count} - {name_changed_band}: {location}')
                    origin_list.append(location)
            except:
                print(f'{count} - {name_changed_band}: {location}')
                origin_list.append(location)      
        else:
            try:
                url = f"https://en.wikipedia.org/wiki/{name_changed}"
                response = requests.get(url).content
                soup = BeautifulSoup(response, "html.parser")

                origin = soup.select('table tr th', class_='infobox-label')

                if len(origin) > 0:
                    if origin[2].text == 'Origin':
                        location = soup.select('table tr td', class_='infobox-data')[1].text

                        if 'This article' in location:
                            location = soup.select('table tr td', class_='infobox-data')[3].text        
                            print(f'{count} - {name_changed_band}: {location}')
                            origin_list.append(location)
                        else:
                            print(f'{count} - {name_changed_band}: {location}')
                            origin_list.append(location)

                    elif origin[3].text == 'Origin':
                        location = soup.select('table tr td', class_='infobox-data')[2].text
                        print(f'{count} - {name_changed_band}: {location}')
                        origin_list.append(location) 

                    else:
                        print(f'{count} - {index}: no location found')
                        origin_list.append(np.nan)  
                else:
                    print(f'{count} - {index}: short length')
                    origin_list.append(np.nan)
            except:
                print(f'{count} - {index}: error')
                origin_list.append(np.nan)
    except:
        print(f'{count} - {index}: error')
        origin_list.append(np.nan)

    if len(artists_list) != len(origin_list):
        print('different lengths')
        break

    count+=1

0 - Less_Than_Jake_(band): Gainesville, Florida, U.S.
1 - Sparta_(band): El Paso, Texas
2 - Muse_(band): Teignmouth, Devon, England
3 - Finch: no location found
4 - Queens_of_the_Stone_Age_(band): Seattle, Washington, U.S.
5 - Transplants_(band): Los Angeles, California, United States[1]
6 - Dream_Theater_(band): Boston, Massachusetts, United States
7 - A_Perfect_Circle_(band): Los Angeles, California, U.S.
8 - Rooney_(band): Los Angeles, California, U.S.
9 - AFI_(band): Ukiah, California, U.S.


In [ ]:
# 107 - Emery_(band): This article needs additional citations for verification. Please help improve this article by adding citations to reliable sources. Unsourced material may be challenged and removed.Find sources: "Emery" band – news · newspapers · books · scholar · JSTOR (June 2008) (Learn how and when to remove this message)
# 142 - Millencolin_(band): This article needs additional citations for verification. Please help improve this article by adding citations to reliable sources. Unsourced material may be challenged and removed.Find sources: "Millencolin" – news · newspapers · books · scholar · JSTOR (November 2016) (Learn how and when to remove this message)
# 156 - Sugarcult_(band): This biography of a living person needs additional citations for verification. Please help by adding reliable sources. Contentious material about living persons that is unsourced or poorly sourced must be removed immediately from the article and its talk page, especially if potentially libelous.Find sources: "Sugarcult" – news · newspapers · books · scholar · JSTOR (November 2020) (Learn how and when to remove this message)

### **Testing code for strange cases**

In [ ]:
df = pd.read_csv('Datasets/df_ratings.csv')
artists = df['artist'].unique()

artists_list = []
origin_list = []
count=0

for index in artists[140:150]:

    artists_list.append(index)
    name_changed = index.replace(' ', '_')
    name_changed_band = name_changed + ('_(band)')

    try:
        url = f"https://en.wikipedia.org/wiki/{name_changed_band}"
        response = requests.get(url).content
        soup = BeautifulSoup(response, "html.parser")

        origin = soup.select('table tr th', class_='infobox-label')

        if len(origin) > 0:
            try:
                if origin[2].text == 'Origin':
                    location = soup.select('table tr td', class_='infobox-data')[1].text
                elif origin[3].text == 'Origin':
                    location = soup.select('table tr td', class_='infobox-data')[2].text
                # else:
                    
                if 'multiple issues' in location:
                    location = soup.select('table tr td', class_='infobox-data')[7].text        
                    print(f'{count} - {name_changed_band}: multiple issues - {location}')
                    origin_list.append(location)
                elif 'additional citations' in location:
                    location = soup.select('table tr td', class_='infobox-data')[3].text        
                    print(f'{count} - {name_changed_band}: additional citations - {location}')
                    origin_list.append(location)

                else:
                    print(f'{count} - {name_changed_band}: {location}')
                    origin_list.append(location)
            except:
                print(f'{count} - {name_changed_band}: {location}')
                origin_list.append(location)      
        else:
            try:
                url = f"https://en.wikipedia.org/wiki/{name_changed}"
                response = requests.get(url).content
                soup = BeautifulSoup(response, "html.parser")

                origin = soup.select('table tr th', class_='infobox-label')

                if len(origin) > 0:
                    if origin[2].text == 'Origin':
                        location = soup.select('table tr td', class_='infobox-data')[1].text

                        if 'multiple issues' in location:
                            location = soup.select('table tr td', class_='infobox-data')[7].text        
                            print(f'{count} - {name_changed_band}: multiple issues - {location}')
                            origin_list.append(location)
                        elif 'additional citations' in location:
                            location = soup.select('table tr td', class_='infobox-data')[3].text        
                            print(f'{count} - {name_changed_band}: additional citations - {location}')
                            origin_list.append(location)
                        else:
                            print(f'{count} - {name_changed_band}: {location}')
                            origin_list.append(location)

                    elif origin[3].text == 'Origin':
                        location = soup.select('table tr td', class_='infobox-data')[2].text
                        print(f'{count} - {name_changed_band}: {location}')
                        origin_list.append(location) 

                    else:
                        print(f'{count} - {index}: no location found')
                        origin_list.append(np.nan)  
                else:
                    print(f'{count} - {index}: short length')
                    origin_list.append(np.nan)
            except:
                print(f'{count} - {index}: error')
                origin_list.append(np.nan)
    except:
        print(f'{count} - {index}: error')
        origin_list.append(np.nan)

    if len(artists_list) != len(origin_list):
        print('different lengths')
        break

    count+=1

0 - Funeral_for_a_Friend_(band): Bridgend, Wales
1 - Millencolin_(band): multiple issues - Örebro, Sweden
2 - The_Flaming_Lips_(band): Oklahoma City, Oklahoma, U.S.
3 - Feeder_(band): Feeder in 2008
4 - Descendents_(band): Manhattan Beach, California, U.S.
5 - PJ Harvey: no location found
6 - Godsmack_(band): Lawrence, Massachusetts U.S.
7 - Blind_Faith_(band): Ripley, Surrey, England
8 - Van_Halen_(band): Pasadena, California, U.S.
9 - Damageplan_(band): Dallas, Texas, U.S.


## **Simplified code**

In [ ]:
df_ratings = pd.read_csv('Datasets/df_ratings.csv')
artists = df_ratings['artist'].unique()

artists_list = []
origin_list = []
count=0

for index in artists[30:100]:

    artists_list.append(index)
    name_changed = index.replace(' ', '_')
    name_changed_band = name_changed + ('_(band)')

    try:
        url = f"https://en.wikipedia.org/wiki/{name_changed_band}"
        response = requests.get(url).content
        soup = BeautifulSoup(response, "html.parser")

        table = soup.select('#mw-content-text > div.mw-content-ltr.mw-parser-output > table.infobox')

        location = table[0].text.split('Origin')[1].split('Genres')[0]
        print(f'{count} - {name_changed_band}: {location}')
        origin_list.append(location)

    except:
        try:
            url = f"https://en.wikipedia.org/wiki/{name_changed}"
            response = requests.get(url).content
            soup = BeautifulSoup(response, "html.parser")

            table = soup.select('#mw-content-text > div.mw-content-ltr.mw-parser-output > table.infobox')

            try:
                location = table[0].text.split('Origin')[1].split('Genres')[0]
                print(f'{count} - {name_changed}: {location}')
                origin_list.append(location)
            except:
                location = table[0].text.split(')')[2].split('Genres')[0]
                print(f'{count} - individual {name_changed}: {location}')
                origin_list.append(location)
        except:
            try:
                url = f"https://es.wikipedia.org/wiki/{name_changed}"
                response = requests.get(url).content
                soup = BeautifulSoup(response, "html.parser")

                table = soup.select('#mw-content-text > div.mw-content-ltr.mw-parser-output > table.infobox')

                location = table[0].text.split('Origen\n')[1].split(' Información')[0]
                print(f'{count} - {name_changed} (español): {location}')
                origin_list.append(location)

            except:
                print(f'{count} - {index}: error')
                origin_list.append(np.nan)

    if len(artists_list) != len(origin_list):
        print('different lengths')
        break

    count+=1

0 - Yes_(band): London, England
1 - Brainiac_(band): Dayton, Ohio, United States
2 - Rise_Against: Chicago, Illinois, U.S.
3 - Pearl_Jam_(band): Seattle, Washington, U.S.
4 - individual Joe_Satriani: Westbury, New York, U.S.
5 - Iron_Maiden_(band): London, England
6 - Streetlight_Manifesto: New Brunswick, New Jersey, United States
7 - O.A.R._(band): Rockville, Maryland, United States
8 - The_Beatles_(band): Liverpool, England
9 - Metallica_(band): Los Angeles, California, U.S.


## **Saving the data into a dataframe**

In [308]:
df_locations = pd.DataFrame({'artist': artists_list
                             , 'origin': origin_list})
df_locations.shape

(10, 2)

In [309]:
df_locations

,artist,origin
0,Yes,"London, England"
1,Brainiac,"Dayton, Ohio, United States"
2,Rise Against,"Chicago, Illinois, U.S."
3,Pearl Jam,"Seattle, Washington, U.S."
4,Joe Satriani,"Westbury, New York, U.S."
5,Iron Maiden,"London, England"
6,Streetlight Manifesto,"New Brunswick, New Jersey, United States"
7,O.A.R.,"Rockville, Maryland, United States"
8,The Beatles,"Liverpool, England"
9,Metallica,"Los Angeles, California, U.S."


In [310]:
df_locations.sample(10)

,artist,origin
5,Iron Maiden,"London, England"
4,Joe Satriani,"Westbury, New York, U.S."
2,Rise Against,"Chicago, Illinois, U.S."
3,Pearl Jam,"Seattle, Washington, U.S."
7,O.A.R.,"Rockville, Maryland, United States"
6,Streetlight Manifesto,"New Brunswick, New Jersey, United States"
9,Metallica,"Los Angeles, California, U.S."
0,Yes,"London, England"
8,The Beatles,"Liverpool, England"
1,Brainiac,"Dayton, Ohio, United States"


## **Exporting to csv**

In [311]:
# df_locations.to_csv('Datasets/df_locations.csv', index=False)

In [312]:
# df_locations = pd.read_csv('Datasets/df_locations.csv')

## **Null values**

In [313]:
df_locations['origin'].isna().sum()

0

In [314]:
# percentage of nulls
print(f'{(df_locations['origin'].isna().sum() / df_locations.shape[0])*100} % of nulls')

0.0 % of nulls


In [315]:
df_locations[df_locations['origin'].isna()]

,artist,origin


Let's search for these bands manually

In [316]:
# df_locations.loc[3, 'origin'] = 'Temecula, California, Estados Unidos'

In [317]:
df_locations['origin'].isna().sum()

0

## **Create ``city``**

In [318]:
df_locations['city'] = df_locations['origin'].apply(lambda x: str(x).split(', ')[0])
df_locations

,artist,origin,city
0,Yes,"London, England",London
1,Brainiac,"Dayton, Ohio, United States",Dayton
2,Rise Against,"Chicago, Illinois, U.S.",Chicago
3,Pearl Jam,"Seattle, Washington, U.S.",Seattle
4,Joe Satriani,"Westbury, New York, U.S.",Westbury
5,Iron Maiden,"London, England",London
6,Streetlight Manifesto,"New Brunswick, New Jersey, United States",New Brunswick
7,O.A.R.,"Rockville, Maryland, United States",Rockville
8,The Beatles,"Liverpool, England",Liverpool
9,Metallica,"Los Angeles, California, U.S.",Los Angeles


In [319]:
print(f'{df_locations['origin'].nunique()} addresses')
print(f'{df_locations['city'].nunique()} cities')

9 addresses
9 cities


In [320]:
df_locations.groupby(['city', 'origin']).agg('count').sort_values(['city'])

,,artist
city,origin,
Chicago,"Chicago, Illinois, U.S.",1
Dayton,"Dayton, Ohio, United States",1
Liverpool,"Liverpool, England",1
London,"London, England",2
Los Angeles,"Los Angeles, California, U.S.",1
New Brunswick,"New Brunswick, New Jersey, United States",1
Rockville,"Rockville, Maryland, United States",1
Seattle,"Seattle, Washington, U.S.",1
Westbury,"Westbury, New York, U.S.",1


## Replace dots

In [ ]:
df_locations.sort_values('origin')

,artist,origin
6,Dream Theater,"Boston, Massachusetts, United States"
1,Sparta,"El Paso, Texas"
0,Less Than Jake,"Gainesville, Florida, US"
7,A Perfect Circle,"Los Angeles, California, US"
8,Rooney,"Los Angeles, California, US"
5,Transplants,"Los Angeles, California, United States[1]"
4,Queens of the Stone Age,"Seattle, Washington, US"
2,Muse,"Teignmouth, Devon, England"
9,AFI,"Ukiah, California, US"
3,Finch,NaN


In [ ]:
# replace special characters for spaces
df_locations['origin'] = df_locations['origin'].str.replace('.', '')

# show the df to check it has been changed correctly
df_locations.sort_values('origin')

,artist,origin
6,Dream Theater,"Boston, Massachusetts, United States"
1,Sparta,"El Paso, Texas"
0,Less Than Jake,"Gainesville, Florida, US"
7,A Perfect Circle,"Los Angeles, California, US"
8,Rooney,"Los Angeles, California, US"
5,Transplants,"Los Angeles, California, United States1"
4,Queens of the Stone Age,"Seattle, Washington, US"
2,Muse,"Teignmouth, Devon, England"
9,AFI,"Ukiah, California, US"
3,Finch,NaN


In [ ]:
# exporting to csv
df_locations.to_csv('Datasets/df_locations.csv', index=False)

## **GeoPy**

In [321]:
geolocator = Nominatim(user_agent="music_analysis")

unique_cities = df_locations['city'].unique()

address_list = []
country_list = []
latitude_list = []
longitude_list = []
count = 0

for city in unique_cities:
  count+=1
  time.sleep(1)
  location = geolocator.geocode(city)

  print(f'{count}/{len(unique_cities)} - {location.address}')  

# save the info in lists
  address_list.append(location.address)
  country_list.append(location.address.split(', ')[-1])
  latitude_list.append(location.latitude)
  longitude_list.append(location.longitude)

1/9 - London, Greater London, England, United Kingdom
2/9 - Dayton, Montgomery County, Ohio, United States
3/9 - Chicago, Cook County, Illinois, United States
4/9 - Seattle, King County, Washington, United States
5/9 - Village of Westbury, Town of North Hempstead, Nassau County, New York, United States
6/9 - New Brunswick / Nouveau-Brunswick, Canada
7/9 - Rockville, Montgomery County, Maryland, United States
8/9 - Liverpool, Liverpool City Region, England, United Kingdom
9/9 - Los Angeles, Los Angeles County, California, United States


### **Save it into a dataframe**

In [322]:
df = pd.DataFrame({'country': country_list
                               , 'city': unique_cities
                               , 'latitude': latitude_list
                               , 'longitude': longitude_list
                               , 'address': address_list})
df

,country,city,latitude,longitude,address
0,United Kingdom,London,51.489334,-0.144055,"London, Greater London, England, United Kingdom"
1,United States,Dayton,39.758948,-84.191607,"Dayton, Montgomery County, Ohio, United States"
2,United States,Chicago,41.875562,-87.624421,"Chicago, Cook County, Illinois, United States"
3,United States,Seattle,47.603832,-122.330062,"Seattle, King County, Washington, United States"
4,United States,Westbury,40.755656,-73.587627,"Village of Westbury, Town of North Hempstead, ..."
5,Canada,New Brunswick,46.500283,-66.750183,"New Brunswick / Nouveau-Brunswick, Canada"
6,United States,Rockville,39.081798,-77.151684,"Rockville, Montgomery County, Maryland, United..."
7,United Kingdom,Liverpool,53.407199,-2.991680,"Liverpool, Liverpool City Region, England, Uni..."
8,United States,Los Angeles,34.053691,-118.242766,"Los Angeles, Los Angeles County, California, U..."


In [ ]:
# save it in a csv file
# df_coordinates.to_csv('Datasets/df_coordinates.csv', index=False)

### **Concat with the previous df**

In [323]:
df_coordinates

,country,city,latitude,longitude,address
0,United States,Gainesville,29.651968,-82.324985,"Gainesville, Alachua County, Florida, United S..."
1,United States,El Paso,31.760116,-106.487040,"El Paso, El Paso County, Texas, United States"
2,United Kingdom,Teignmouth,50.546338,-3.495780,"Teignmouth, Teignbridge, Devon, England, Unite..."
3,United States,Temecula,33.494635,-117.147366,"Temecula, Riverside County, California, United..."
4,United States,Seattle,47.603832,-122.330062,"Seattle, King County, Washington, United States"
5,United States,Los Angeles,34.053691,-118.242766,"Los Angeles, Los Angeles County, California, U..."
6,United States,Boston,42.355433,-71.060511,"Boston, Suffolk County, Massachusetts, United ..."
7,United States,Ukiah,39.150166,-123.207786,"Ukiah, Mendocino County, California, 95482, Un..."
8,United States,Long Beach,33.769016,-118.191604,"Long Beach, Los Angeles County, California, Un..."
9,United Kingdom,Crawley,51.110344,-0.180109,"Crawley, West Sussex, England, United Kingdom"


In [324]:
# import the last df
df_coordinates_scrapped = pd.read_csv('Datasets/df_coordinates.csv')
df_coordinates_scrapped.shape

(15, 5)

In [325]:
# concat with the df of the locations I just got
df_coordinates = pd.concat([df_coordinates_scrapped, df])
df_coordinates.reset_index(drop=True, inplace=True)
df_coordinates.shape

(24, 5)

In [326]:
def check_duplicates(df):
    if df['latitude'].duplicated().sum() == 0:
        print('No duplicates')
    else: 
        duplicates = df['latitude'].duplicated().sum()
        print(f'Found {duplicates} duplicates:')
        print(f'{df[df['latitude'].duplicated()][['city', 'country']]}\n')

        df.drop_duplicates(subset='latitude', inplace=True)
        df.reset_index(drop=True, inplace=True)
        print(f'Resulting dataset: {df.shape}')

In [327]:
check_duplicates(df_coordinates)

Found 2 duplicates:
           city        country
18      Seattle  United States
23  Los Angeles  United States

Resulting dataset: (22, 5)


In [328]:
# save it in a csv file
df_coordinates.to_csv('Datasets/df_coordinates.csv', index=False)

## **Statistics**

In [329]:
df_coordinates['country'].value_counts()

country
United States     15
United Kingdom     5
Sverige            1
Canada             1
Name: count, dtype: int64

### Merge with the previous df

In [331]:
# merge with the previous df with the artists
df_locations_merged = pd.merge(df_coordinates, df_coordinates, on=['city'])
df_locations_merged

,country_x,city,latitude_x,longitude_x,address_x,country_y,latitude_y,longitude_y,address_y
0,United States,Gainesville,29.651968,-82.324985,"Gainesville, Alachua County, Florida, United S...",United States,29.651968,-82.324985,"Gainesville, Alachua County, Florida, United S..."
1,United States,El Paso,31.760116,-106.487040,"El Paso, El Paso County, Texas, United States",United States,31.760116,-106.487040,"El Paso, El Paso County, Texas, United States"
2,United Kingdom,Teignmouth,50.546338,-3.495780,"Teignmouth, Teignbridge, Devon, England, Unite...",United Kingdom,50.546338,-3.495780,"Teignmouth, Teignbridge, Devon, England, Unite..."
3,United States,Temecula,33.494635,-117.147366,"Temecula, Riverside County, California, United...",United States,33.494635,-117.147366,"Temecula, Riverside County, California, United..."
4,United States,Seattle,47.603832,-122.330062,"Seattle, King County, Washington, United States",United States,47.603832,-122.330062,"Seattle, King County, Washington, United States"
5,United States,Los Angeles,34.053691,-118.242766,"Los Angeles, Los Angeles County, California, U...",United States,34.053691,-118.242766,"Los Angeles, Los Angeles County, California, U..."
6,United States,Boston,42.355433,-71.060511,"Boston, Suffolk County, Massachusetts, United ...",United States,42.355433,-71.060511,"Boston, Suffolk County, Massachusetts, United ..."
7,United States,Ukiah,39.150166,-123.207786,"Ukiah, Mendocino County, California, 95482, Un...",United States,39.150166,-123.207786,"Ukiah, Mendocino County, California, 95482, Un..."
8,United States,Long Beach,33.769016,-118.191604,"Long Beach, Los Angeles County, California, Un...",United States,33.769016,-118.191604,"Long Beach, Los Angeles County, California, Un..."
9,United Kingdom,Crawley,51.110344,-0.180109,"Crawley, West Sussex, England, United Kingdom",United Kingdom,51.110344,-0.180109,"Crawley, West Sussex, England, United Kingdom"


In [332]:
df_ratings

,album_id,artist,album,rating,votes
0,37,Less Than Jake,Losing Streak,3.90,414
1,40,Sparta,Wiretap Scars,3.79,431
2,41,Muse,Absolution,3.99,4411
3,42,Muse,Showbiz,3.50,2181
4,45,Finch,What It Is to Burn,3.69,864
...,...,...,...,...,...
76737,224795,Echo and The Bunnymen,Ballyhoo: The Best Of Echo And The Bunnymen,4.00,2
76738,224796,SECONDWALL,Over,2.50,1
76739,224797,Simple Plan,Get Your Heart On - The Second Coming!,2.39,9
76740,224799,The Patient Zeros,Sirens Calling,3.50,1


### Selenium

In [ ]:
url = "https://en.wikipedia.org/wiki/"

PATH = ("C:/Program Files (x86)/chromedriver.exe")
driver = webdriver.Chrome(PATH)
driver.get(url)

time.sleep(2)

search = driver.select('')

NameError: name 'time' is not defined